## Proyecto: Turismo de los Alpes  - Grupo 34
Felipe Nuñez
<br>
Jeronimo Vargas Rendon - 202113305
<br>
Estudiante 1: Juan Manuel Pérez - 202021827

In [82]:
# Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Jeronimo
[nltk_data]     Vargas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Cargar Datos

In [83]:
# Carga de datos
df = pd.read_csv('tipo1_entrenamiento_estudiantes.csv')

## 1. Entendimiento de los datos

In [84]:
# Visualización de los primeros registros
print(df.head())

# Total de registros
print("Total de registros:", df.shape[0])

# Verificación de valores faltantes
print(df.isnull().sum())

#Contar duplicados
print("Duplicados:" , df.duplicated().sum())

                                              Review  Class
0  Nos alojamos en una casa alquilada en la ciuda...      4
1  La comida está bien, pero nada especial. Yo te...      3
2  En mi opinión, no es una como muchos usuarios ...      3
3  esta curiosa forma que asemeja una silla de mo...      4
4  Lo mejor era la limonada. Me gusto la comida d...      2
Total de registros: 7875
Review    0
Class     0
dtype: int64
Duplicados: 71


## 2. Preparación de los datos

In [85]:
# Imprimir df
print(df.columns)

Index(['Review', 'Class'], dtype='object')


In [86]:
# Eliminar duplicados
df = df.drop_duplicates()

In [87]:
import re
def remove_noise(text):
    text = re.sub(r'<.*?>', '', text)  # Eliminar HTML tags
    text = re.sub(r'[0-9]+', '', text)  # Eliminar números
    text = re.sub(r'[^a-záéíóúñA-ZÁÉÍÓÚÑ\s]', '', text)  # Eliminar caracteres especiales
    return text

df['Review'] = df['Review'].apply(remove_noise)

In [88]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descarga de los recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Definición de las stopwords en español
stop_words = set(stopwords.words('spanish'))

def clean_text(text):
    # Tokenización del texto
    words = word_tokenize(text.lower())
    # Filtrado de palabras que no son stopwords y son alfabéticas
    filtered_words = [word for word in words if word not in stop_words and word.isalpha()]
    # Reconstrucción del texto
    return " ".join(filtered_words)

# Asumiendo que 'Review' es la columna con los textos a limpiar
df['Review'] = df['Review'].apply(clean_text)

[nltk_data] Downloading package stopwords to C:\Users\Jeronimo
[nltk_data]     Vargas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Jeronimo
[nltk_data]     Vargas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
import spacy # type: ignore
nlp = spacy.load('es_core_news_sm')
def normalize_text(text):
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)
df['Review'] = df['Review'].apply(normalize_text)


c:\Users\Jeronimo Vargas\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [90]:
# Función para preprocesar el texto
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(lemmatized_words)


# Aplicación de la función de preprocesamiento
df['processed_text'] = df['Review'].apply(preprocess_text)

# Vectorización
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['processed_text']).toarray()

"""
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Utiliza bigramas además de unigramas
X = vectorizer.fit_transform(df['Review'])
"""

# Extracción etiquetas
y = df['Class']


## 3. Modelo

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
# Regression logistica
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

modelLR = LogisticRegression()
modelLR.fit(X_train, y_train)
y_pred = modelLR.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


              precision    recall  f1-score   support

           1       0.51      0.39      0.44       160
           2       0.43      0.34      0.38       240
           3       0.39      0.34      0.36       312
           4       0.38      0.41      0.39       397
           5       0.57      0.68      0.62       452

    accuracy                           0.46      1561
   macro avg       0.45      0.43      0.44      1561
weighted avg       0.46      0.46      0.45      1561

Accuracy: 0.4612427930813581


In [93]:
# Modelo multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB

modelNB = MultinomialNB()
modelNB.fit(X_train, y_train)
y_pred = modelNB.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


              precision    recall  f1-score   support

           1       0.56      0.24      0.33       160
           2       0.41      0.38      0.39       240
           3       0.38      0.28      0.32       312
           4       0.38      0.40      0.39       397
           5       0.53      0.74      0.62       452

    accuracy                           0.45      1561
   macro avg       0.45      0.41      0.41      1561
weighted avg       0.45      0.45      0.44      1561

Accuracy: 0.4535554131966688


In [94]:
# Modelo Random Forest
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()
modelRF.fit(X_train, y_train)
y_pred = modelRF.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")



              precision    recall  f1-score   support

           1       0.46      0.19      0.27       160
           2       0.42      0.43      0.42       240
           3       0.38      0.29      0.33       312
           4       0.39      0.38      0.39       397
           5       0.53      0.72      0.61       452

    accuracy                           0.45      1561
   macro avg       0.44      0.40      0.40      1561
weighted avg       0.44      0.45      0.43      1561

Accuracy: 0.4509929532351057


## 4. Evaluacion

### Predicciones Archivo de Prueba

In [97]:
# Predicciones con el modelo a los siguientes Datos usando el mejor modelo LR

df_pred = pd.read_csv('particion_prueba_estudiantes.csv')

df_p = df_pred.copy()

# Predecir con el modleo de Liner Regression
df_p['Review'] = df_p['Review'].apply(remove_noise)
df_p['Review'] = df_p['Review'].apply(clean_text)
df_p['Review'] = df_p['Review'].apply(normalize_text)
df_p['Review'] = df_p['Review'].apply(preprocess_text)
X_pred = vectorizer.transform(df_p['Review']).toarray()
y_pred = modelLR.predict(X_pred)
df_p['Class'] = y_pred

# Guardar predicciones
df_pred['Class'] = y_pred
df_pred.to_csv('predicciones_estudiantes.csv', index=False)
print(df_p.head())





                                              Review  Class
0  primero noche encontrar habitación nido cucara...      1
1  calle catedral platillo tradicional tipo gourm...      4
2  porción miserable agua sabor cloro distraído m...      2
3  cartagena encantar todo ciudad colonial visita...      5
4  ibar mucho ilusion disfrutar espectaculo luz s...      3


## 5. Resultados